# Map-reduce in pure python #

Let's get comfortable with map reduce in pure python (no Spark!).

We're going to define our own map-reduce functions, then use python `for` loops to execute them. We'll inspect the output at each stage.

Here are functions for counting letters in a string.

In [1]:
def my_map_function(x):
    return (x, 1)

def my_key_function(x):
    return x[0]

def my_reduce_function(x, y):
    return (x[0], x[1]+y[1])

Here's a nice string.

In [2]:
my_list = list("a man a plan a canal panama")
my_list

['a',
 ' ',
 'm',
 'a',
 'n',
 ' ',
 'a',
 ' ',
 'p',
 'l',
 'a',
 'n',
 ' ',
 'a',
 ' ',
 'c',
 'a',
 'n',
 'a',
 'l',
 ' ',
 'p',
 'a',
 'n',
 'a',
 'm',
 'a']

Let's plug those into code that executes a map reduce.

In [4]:
new_list = []
for element in my_list:
    new_list.append(
        my_map_function(element)
    )
new_list

[('a', 1),
 (' ', 1),
 ('m', 1),
 ('a', 1),
 ('n', 1),
 (' ', 1),
 ('a', 1),
 (' ', 1),
 ('p', 1),
 ('l', 1),
 ('a', 1),
 ('n', 1),
 (' ', 1),
 ('a', 1),
 (' ', 1),
 ('c', 1),
 ('a', 1),
 ('n', 1),
 ('a', 1),
 ('l', 1),
 (' ', 1),
 ('p', 1),
 ('a', 1),
 ('n', 1),
 ('a', 1),
 ('m', 1),
 ('a', 1)]

In [5]:
reducers = {}
for element in new_list:
    key = my_key_function(element)
    
    if not key in reducers:
        reducers[key] = []
    
    reducers[key].append(element)

for key, value in reducers.items():
    print key, ':', value

a : [('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1)]
  : [(' ', 1), (' ', 1), (' ', 1), (' ', 1), (' ', 1), (' ', 1)]
c : [('c', 1)]
m : [('m', 1), ('m', 1)]
l : [('l', 1), ('l', 1)]
n : [('n', 1), ('n', 1), ('n', 1), ('n', 1)]
p : [('p', 1), ('p', 1)]


In [6]:
reducer_results = {}
for key, value_list in reducers.items():
#     print key, value_list
    reducer_results[key] = value_list[0]
    for value in value_list[1:]:
#         print value
        reducer_results[key] = my_reduce_function(reducer_results[key], value)

for key, value in reducer_results.items():
    print key, ':', value

a : ('a', 10)
  : (' ', 6)
c : ('c', 1)
m : ('m', 2)
l : ('l', 2)
n : ('n', 4)
p : ('p', 2)


In [7]:
final_results = reducer_results.values()
final_results

[('a', 10), (' ', 6), ('c', 1), ('m', 2), ('l', 2), ('n', 4), ('p', 2)]

Let's put that all in a single function:

In [8]:
def execute_map_reduce(my_map_function, my_key_function, my_reduce_function, my_list, print_output=True):
    if print_output:
        print "===== Raw data ====="
        print my_list


    new_list = []
    for element in my_list:
        new_list.append(
            my_map_function(element)
        )
    if print_output:
        print "===== After mapping ====="
        print new_list

    reducers = {}
    for element in new_list:
        key = my_key_function(element)

        if not key in reducers:
            reducers[key] = []

        reducers[key].append(element)

    if print_output:
        print "===== Grouped for reducing ====="
        for key, value in reducers.items():
            print key, ':', value

    reducer_results = {}
    for key, value_list in reducers.items():
    #     print key, value_list
        reducer_results[key] = value_list[0]
        for value in value_list[1:]:
    #         print value
            reducer_results[key] = my_reduce_function(reducer_results[key], value)

    if print_output:
        print "===== After reducing ====="
        for key, value in reducer_results.items():
            print key, ':', value


    final_results = reducer_results.values()
    if print_output:
        print "===== Final results ====="
        print final_results
    
    return final_results

Test it out:

In [9]:
output = execute_map_reduce(my_map_function, my_key_function, my_reduce_function, my_list)

===== Raw data =====
['a', ' ', 'm', 'a', 'n', ' ', 'a', ' ', 'p', 'l', 'a', 'n', ' ', 'a', ' ', 'c', 'a', 'n', 'a', 'l', ' ', 'p', 'a', 'n', 'a', 'm', 'a']
===== After mapping =====
[('a', 1), (' ', 1), ('m', 1), ('a', 1), ('n', 1), (' ', 1), ('a', 1), (' ', 1), ('p', 1), ('l', 1), ('a', 1), ('n', 1), (' ', 1), ('a', 1), (' ', 1), ('c', 1), ('a', 1), ('n', 1), ('a', 1), ('l', 1), (' ', 1), ('p', 1), ('a', 1), ('n', 1), ('a', 1), ('m', 1), ('a', 1)]
===== Grouped for reducing =====
a : [('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1), ('a', 1)]
  : [(' ', 1), (' ', 1), (' ', 1), (' ', 1), (' ', 1), (' ', 1)]
c : [('c', 1)]
m : [('m', 1), ('m', 1)]
l : [('l', 1), ('l', 1)]
n : [('n', 1), ('n', 1), ('n', 1), ('n', 1)]
p : [('p', 1), ('p', 1)]
===== After reducing =====
a : ('a', 10)
  : (' ', 6)
c : ('c', 1)
m : ('m', 2)
l : ('l', 2)
n : ('n', 4)
p : ('p', 2)
===== Final results =====
[('a', 10), (' ', 6), ('c', 1), ('m', 2), ('l', 2), ('n', 4), (

In [10]:
execute_map_reduce(
    my_map_function,
    my_key_function,
    my_reduce_function,
    "The quick brown fox jumped over the lazy dog",
    print_output=False
)

[(' ', 8),
 ('T', 1),
 ('a', 1),
 ('c', 1),
 ('b', 1),
 ('e', 4),
 ('d', 2),
 ('g', 1),
 ('f', 1),
 ('i', 1),
 ('h', 2),
 ('k', 1),
 ('j', 1),
 ('m', 1),
 ('l', 1),
 ('o', 4),
 ('n', 1),
 ('q', 1),
 ('p', 1),
 ('r', 2),
 ('u', 2),
 ('t', 1),
 ('w', 1),
 ('v', 1),
 ('y', 1),
 ('x', 1),
 ('z', 1)]

Let's try a different mapper.

In [11]:
def lowercase_map_function(x):
    return (x.lower(), 1)

print 'With the old mapper:'
print execute_map_reduce(my_map_function, my_key_function, my_reduce_function, "AAAaaa", print_output=False)

print
print 'With the lowercase mapper:'
print execute_map_reduce(lowercase_map_function, my_key_function, my_reduce_function, "AAAaaa", print_output=False)


With the old mapper:
[('A', 3), ('a', 3)]

With the lowercase mapper:
[('a', 6)]


### Exercise 1 ###

Take a look a these map-reduce functions. What are they doing?

- With a neighbor, explain what these functions are doing.
- What is the expected output for each of these cases? Write your guesses before running the code.
    - "AAAAa"
    - "BBBBbbb"
    - "abbcccddddeeeeeffffffggggggghhhhhhhhiiiiiiiiijjjjjjjjjj"
    - "a man a plan a canal panama"

In [13]:
def exercise_1_mapper(x):
    is_vowel = x in list("aeiouyAEIOUY")
    return (x, (is_vowel, 2))

def exercise_1_reducer(x, y):
    print x, y
    if x[1][0]:
        return (x[0], (x[1][0], x[1][1]*y[1][1]))
    else:
        return (x[0], (x[1][0], x[1][1]+y[1][1]))

_ = execute_map_reduce(
    exercise_1_mapper,
    my_key_function,
    exercise_1_reducer,
    "aaabbb",
#     print_output=False
)


===== Raw data =====
aaabbb
===== After mapping =====
[('a', (True, 2)), ('a', (True, 2)), ('a', (True, 2)), ('b', (False, 2)), ('b', (False, 2)), ('b', (False, 2))]
===== Grouped for reducing =====
a : [('a', (True, 2)), ('a', (True, 2)), ('a', (True, 2))]
b : [('b', (False, 2)), ('b', (False, 2)), ('b', (False, 2))]
('a', (True, 2)) ('a', (True, 2))
('a', (True, 4)) ('a', (True, 2))
('b', (False, 2)) ('b', (False, 2))
('b', (False, 4)) ('b', (False, 2))
===== After reducing =====
a : ('a', (True, 8))
b : ('b', (False, 6))
===== Final results =====
[('a', (True, 8)), ('b', (False, 6))]


### Exercise 2 ###

Write new map-reduce functions so that uppercase letters count for 5 points, and lowercase letters only count for 1.
```
"a" => [(a, 1),(b,2)]
"A" => [(a, 5),(b,2)]
"AaBbB" => [(a, 2),(b,2)]
```

Hint: One of our existing reducers cover this case, so you only need to rewrite the mapper.


In [16]:
def exercise_1_mapper(x):
    is_uppercase_letter = x.isupper()
    if(is_uppercase_letter):
        return (x.lower(), (is_uppercase_letter, 5))
    else:
        return (x, (is_uppercase_letter, 1))

def exercise_1_reducer(x, y):
    print x, y
    #if x[1][0]:
        #return (x[0], (x[1][0], x[1][1]*y[1][1]))
    #else:
    return (x[0], (x[1][0], x[1][1]+y[1][1]))

_ = execute_map_reduce(
    exercise_1_mapper,
    my_key_function,
    exercise_1_reducer,
    "aaaAbbb",
#     print_output=False
)

===== Raw data =====
aaaAbbb
===== After mapping =====
[('a', (False, 1)), ('a', (False, 1)), ('a', (False, 1)), ('a', (True, 5)), ('b', (False, 1)), ('b', (False, 1)), ('b', (False, 1))]
===== Grouped for reducing =====
a : [('a', (False, 1)), ('a', (False, 1)), ('a', (False, 1)), ('a', (True, 5))]
b : [('b', (False, 1)), ('b', (False, 1)), ('b', (False, 1))]
('a', (False, 1)) ('a', (False, 1))
('a', (False, 2)) ('a', (False, 1))
('a', (False, 3)) ('a', (True, 5))
('b', (False, 1)) ('b', (False, 1))
('b', (False, 2)) ('b', (False, 1))
===== After reducing =====
a : ('a', (False, 8))
b : ('b', (False, 3))
===== Final results =====
[('a', (False, 8)), ('b', (False, 3))]


### Exercise 3 ###

The output from exercise 1 has an extra Boolean field.

```
aaabbb => [('a', (True, 8)), ('b', (False, 6))]
```

Can you write another map-reduce function that takes the output of the first map-reduce and gets rid the boolean field?

```
[('a', (True, 8)), ('b', (False, 6))] ==> [('a', 8), ('b', 6)]
```


In [17]:
def exercise_1_mapper(x):
    is_uppercase_letter = x.isupper()
    if(is_uppercase_letter):
        return (x.lower(),  5)
    else:
        return (x,  1)

def exercise_1_reducer(x, y):
    print x, y
    #if x[1][0]:
        #return (x[0], (x[1][0], x[1][1]*y[1][1]))
    #else:
    return (x[0], x[1]+y[1])

_ = execute_map_reduce(
    exercise_1_mapper,
    my_key_function,
    exercise_1_reducer,
    "aaaAbbb",
#     print_output=False
)

===== Raw data =====
aaaAbbb
===== After mapping =====
[('a', 1), ('a', 1), ('a', 1), ('a', 5), ('b', 1), ('b', 1), ('b', 1)]
===== Grouped for reducing =====
a : [('a', 1), ('a', 1), ('a', 1), ('a', 5)]
b : [('b', 1), ('b', 1), ('b', 1)]
('a', 1) ('a', 1)
('a', 2) ('a', 1)
('a', 3) ('a', 5)
('b', 1) ('b', 1)
('b', 2) ('b', 1)
===== After reducing =====
a : ('a', 8)
b : ('b', 3)
===== Final results =====
[('a', 8), ('b', 3)]
